In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Mục đích notebook

Thực hiện biến đổi các đặc trưng và lưu lại để tiện sử dụng sau này.

In [ ]:
%%time
# Tổng số dòng dữ liệu
total = 629145481 
# Số dữ liệu được tính mỗi bước.
chunksize = 150_000

# Tải dữ liệu sử dụng chunk, do nếu tải hết dữ liệu vào trong lúc Editing Noteboong thì sẽ báo lỗi.
chunks = pd.read_csv('../input/LANL-Earthquake-Prediction/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64}, chunksize = chunksize)

c = 1

segments = total // chunksize
# for debug
# segments = 10
    
# Segment là số lượng đoạn được lấy từ input.
# Các chunk được load sau đó tiền xử lý luôn luôn.

In [ ]:
# Hiển thị đến độ chính xác 15 chữ số sau dấu phẩy.
pd.options.display.precision = 15

## Chuyển đổi, làm sạch dữ liệu

Nạp các thư viện.

In [ ]:
%%time
# Load các thư viện liên quan đến biến đổi tín hiệu và thống kê sẽ được sủ dụng.
from scipy import stats
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve

# Load thư viện về hồi quy tuyến tính đơn giản để tìm xu hướng của dữ liệu.
from sklearn.linear_model import LinearRegression
from tqdm import tqdm_notebook as tqdm

import numpy as np

Định nghĩa làm STA và LTA

In [ ]:
%%time
import warnings
warnings.filterwarnings('ignore')

def classic_sta_lta(x, length_sta, length_lta):
    """
    Input: x là một mảng số.
    STA: Short Term Avg.
    LTA: Long Term Avg.
    
    Term Average: được tính bằng trung bình của các phần tử liên tiếp nhau.
    """
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Pad zeros
    sta[:length_lta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny
    
    return sta / lta

Tạo một mảng các đặc trưng. Nội dung của mảng được lấy từ [notebook Độ quan trọng đặc trưng](https://www.kaggle.com/ngodang/int3405e-20-lanl-feature-importance).

In [ ]:
keep_cols = ['change_mean', 'abs_rolling_std_10_percentage_0.8',
       'abs_rolling_std_100_percentage_0.99',
       'abs_rolling_std_50_percentage_0.5',
       'abs_rolling_std_100_percentage_0.7',
       'abs_rolling_std_10_percentage_0.99',
       'abs_rolling_std_100_percentage_0.95',
       'abs_rolling_std_100_percentage_0.8',
       'abs_rolling_std_50_percentage_0.3',
       'abs_rolling_std_50_percentage_0.8',
       'abs_rolling_std_50_percentage_0.4',
       'abs_rolling_std_50_percentage_0.6',
       'abs_rolling_std_50_percentage_0.9',
       'abs_rolling_std_10_percentage_0.85',
       'abs_rolling_std_50_percentage_0.99', 'abs_quantile_0.85',
       'abs_rolling_std_200_percentage_0.95',
       'abs_rolling_std_5000_percentage_0.01',
       'abs_rolling_std_5000_percentage_0.1',
       'abs_rolling_std_5000_percentage_0.85',
       'abs_rolling_std_200_percentage_0.99',
       'abs_rolling_std_10000_percentage_0.4',
       'abs_rolling_std_500_percentage_0.7',
       'abs_rolling_std_200_percentage_0.5',
       'abs_rolling_std_200_percentage_0.8',
       'abs_rolling_std_10000_percentage_0.95',
       'abs_rolling_std_5000_percentage_0.99', 'hmean',
       'abs_rolling_std_50_percentage_0.95',
       'abs_rolling_std_5000_percentage_0.15',
       'abs_rolling_std_5000_percentage_0.2',
       'abs_rolling_std_100_percentage_0.9',
       'abs_rolling_std_10000_percentage_0.15',
       'abs_rolling_std_10000_percentage_0.7',
       'abs_rolling_std_500_percentage_0.9',
       'abs_rolling_std_50_percentage_0.85',
       'abs_rolling_std_10_percentage_0.6', 'rolling_min_std_10000',
       'abs_rolling_std_100_percentage_0.85',
       'abs_rolling_std_5000_percentage_0.7',
       'abs_rolling_std_100_percentage_0.4',
       'abs_rolling_std_50_percentage_0.2',
       'imag_abs_rolling_min_mean_200',
       'abs_rolling_std_500_percentage_0.99',
       'abs_rolling_std_5000_percentage_0.9',
       'abs_rolling_std_10000_percentage_0.1',
       'abs_rolling_std_100_percentage_0.6',
       'abs_rolling_std_200_percentage_0.7', 'abs_quantile_0.5',
       'abs_rolling_std_10_percentage_0.95',
       'abs_rolling_std_200_percentage_0.4',
       'abs_rolling_std_100_percentage_0.5',
       'abs_rolling_std_10_percentage_0.9',
       'abs_rolling_std_500_percentage_0.05',
       'abs_rolling_std_5000_percentage_0.05',
       'abs_rolling_std_5000_percentage_0.3',
       'abs_rolling_std_10000_percentage_0.99', 'real_abs_min',
       'abs_rolling_std_10000_percentage_0.8',
       'imag_abs_rolling_std_50_percentage_0.2',
       'abs_rolling_std_500_percentage_0.8', 'real_dif_mean',
       'abs_rolling_std_5000_percentage_0.5',
       'abs_rolling_std_1000_percentage_0.99',
       'abs_rolling_std_10000_percentage_0.5',
       'abs_rolling_std_5000_percentage_0.4',
       'abs_rolling_std_50_percentage_0.7',
       'abs_rolling_std_200_percentage_0.2',
       'abs_rolling_std_200_percentage_0.15', 'iqr',
       'real_abs_rolling_max_mean_10',
       'abs_rolling_std_10_percentage_0.4',
       'abs_rolling_std_500_percentage_0.6', 'imag_abs_percentile_0.15',
       'abs_rolling_std_500_percentage_0.85',
       'abs_rolling_std_10_percentage_0.5',
       'abs_rolling_std_50_percentage_0.05',
       'abs_rolling_std_200_percentage_0.05',
       'abs_rolling_std_5000_percentage_0.95',
       'abs_rolling_std_100_percentage_0.15',
       'abs_rolling_std_10000_percentage_0.05',
       'abs_rolling_std_1000_percentage_0.9',
       'abs_rolling_std_200_percentage_0.3',
       'abs_rolling_std_500_percentage_0.4', 'imag_hann_window_5000',
       'real_abs_rolling_std_10_percentage_0.01',
       'real_abs_rolling_mean_10000_percentage_0.15',
       'imag_abs_rolling_std_10_percentage_0.01', 'ave_div_0', 'mad',
       'real_abs_percentile_0.85', 'imag_change_mean',
       'abs_rolling_std_100_percentage_0.2', 'imag_skew', 'imag_coef_',
       'ave_div_2', 'gmean', 'real_abs_rolling_max_mean_100',
       'abs_rolling_mean_5000_percentage_0.05', 'imag_ave_div_2',
       'real_rolling_max_ave_10', 'std_div_1',
       'abs_rolling_std_50_percentage_0.15', 'real_abs_percentile_0.1',
       'real_abs_rolling_std_100_percentage_0.2',
       'abs_rolling_std_5000_percentage_0.6', 'imag_rolling_min_ave_5000',
       'min_div_4', 'abs_rolling_std_10000_percentage_0.9',
       'real_abs_rolling_mean_10_percentage_0.01',
       'abs_rolling_max_mean_10000', 'min_max_diff_div_3',
       'real_abs_percentile_0.05', 'max_div_2',
       'real_rolling_max_std_5000', 'abs_rolling_std_50_percentage_0.01',
       'imag_abs_percentile_0.1',
       'real_abs_rolling_std_10_percentage_0.2',
       'abs_rolling_mean_50_percentage_0.4', 'imag_std_div_3']

Thực hiện xử lý dữ liệu và xây dựng đặc trưng. Tuy nhiên, đã bỏ đi các đặc trưng và các nhóm chính không có trong tập 120 đặc trưng được chọn.

In [ ]:
%%time
train = []
y = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

rows = 150_000

for segment in tqdm(range(segments)):
    chunk = next(chunks)
    ad = chunk['acoustic_data']
    y.loc[segment, 'time_to_failure'] = chunk['time_to_failure'].values[-1]
    
    ft = np.fft.fft(ad)
    real_ad = pd.Series(np.real(ft))
    imag_ad = pd.Series(np.imag(ft))
    
    features = dict()
    
    # Tính các feature với ad
    #########################################################################################################
    quantiles = [0.5, 0.85]
    for quantile in quantiles:
        features['abs_quantile_' + str(quantile)] = np.quantile(np.abs(ad), quantile)
        
    div = 5
    for i in range(div):
        features['ave_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].mean()
        features['std_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].std()
        features['max_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].max()
        features['min_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].min()
        features['min_max_diff_div_' + str(i)] = features['max_div_' + str(i)] - features['min_div_' + str(i)]


    windows = [10, 50, 100, 200, 500, 1000, 5000, 10000]
    percentages = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99]
    for window in windows:
        roll_mean = ad.rolling(window).mean().dropna()
        roll_std = ad.rolling(window).std().dropna()

        features['rolling_min_std_' + str(window)] = roll_std.min()
        
        features['abs_rolling_max_mean_' + str(window)] = np.abs(roll_mean).max()
        
        for percentage in percentages:
            features['abs_rolling_mean_{}_percentage_{}'.format(window, percentage)] = np.percentile(roll_mean, percentage)
            features['abs_rolling_std_{}_percentage_{}'.format(window, percentage)] = np.percentile(roll_std, percentage)
            
    dif = ad.diff().dropna().values
    change = (dif / ad.values[:-1])
    change = change[np.nonzero(change)[0]]
    change = change[~np.isnan(change)]
    change = change[change != -np.inf]
    change = change[change != np.inf]
    
    features['change_mean'] = change.mean()
    
    features['hmean'] = stats.hmean(np.abs(ad[ad != 0]))
    features['gmean'] = stats.gmean(np.abs(ad[ad != 0]))
    
    features['iqr'] = np.subtract(*np.percentile(ad, [75, 25]))
    
    features['mad'] = np.mean(np.abs(ad - np.mean(ad)))
    
    del change
    del dif
    
    # Tính các feature với real_ad
    #########################################################################################################
    del ad
    ad = real_ad
    
    prefix = "real_"

    features[prefix + 'abs_min'] = np.abs(ad).min()
        
    features[prefix + 'abs_percentile_' + str(0.05)] = np.percentile(np.abs(ad), 0.05)
    features[prefix + 'abs_percentile_' + str(0.1)] = np.percentile(np.abs(ad), 0.1)
    features[prefix + 'abs_percentile_' + str(0.85)] = np.percentile(np.abs(ad), 0.85)
    
    # Tính các đặc tính theo tổng liên tiếp
    
    roll_mean = ad.rolling(10).mean().dropna()
    features[prefix + 'abs_rolling_max_mean_' + str(10)] = np.abs(roll_mean).max()
    roll_mean = ad.rolling(100).mean().dropna()
    features[prefix + 'abs_rolling_max_mean_' + str(100)] = np.abs(roll_mean).max()
    
    roll_mean = ad.rolling(10).mean().dropna()
    features[prefix + 'abs_rolling_mean_{}_percentage_{}'.format(10, 0.01)] = np.percentile(roll_mean, 0.01)
    roll_mean = ad.rolling(10000).mean().dropna()
    features[prefix + 'abs_rolling_mean_{}_percentage_{}'.format(10000, 0.15)] = np.percentile(roll_mean, 0.15)
    
    roll_std = ad.rolling(10).std().dropna()
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(10, 0.01)] = np.percentile(roll_std, 0.01)
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(10, 0.2)] = np.percentile(roll_std, 0.2)
    roll_std = ad.rolling(100).std().dropna()
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(100, 0.2)] = np.percentile(roll_std, 0.2)
    
    roll_mean = ad.rolling(10).mean().dropna()
    features[prefix + 'rolling_max_ave_' + str(10)] = roll_mean.max()
    roll_std = ad.rolling(5000).std().dropna()
    features[prefix + 'rolling_max_std_' + str(5000)] = roll_std.max()
            
    dif = ad.diff().dropna().values
    features[prefix + 'dif_mean'] = dif.mean()
    
    del dif
    
    # Tính các feature với imag_ad
    #########################################################################################################
    del ad
    ad = imag_ad
    
    prefix = "imag_"
    
    # find trend using linear regression.
    model = LinearRegression().fit(np.arange(len(ad)).reshape(-1, 1), ad)

    features[prefix + 'coef_'] = model.coef_[0]
    
    # divide into smaller segment.
    
    features[prefix + 'ave_div_' + str(2)] = ad[rows // div * 2: rows // div * (2 + 1)].mean()
    features[prefix + 'std_div_' + str(3)] = ad[rows // div * 3: rows // div * (3 + 1)].std()
    
    # some statistic distribution characteristic
    
    features[prefix + 'abs_percentile_' + str(0.1)] = np.percentile(np.abs(ad), 0.1)
    features[prefix + 'abs_percentile_' + str(0.15)] = np.percentile(np.abs(ad), 0.15)
        
    features[prefix + 'skew'] = stats.skew(ad)
    
    features[prefix + 'hann_window_' + str(5000)] = (convolve(ad, hann(5000), mode='same') / sum(hann(5000))).mean()
        
    roll_mean = ad.rolling(5000).mean().dropna()
    features[prefix + 'rolling_min_ave_' + str(5000)] = roll_mean.min()
    
    roll_mean = ad.rolling(200).mean().dropna()
    features[prefix + 'abs_rolling_min_mean_' + str(200)] = np.abs(roll_mean).min()
    
    roll_std = ad.rolling(10).std().dropna()
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(10, 0.01)] = np.percentile(roll_std, 0.01)
    
    roll_std = ad.rolling(50).std().dropna()
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(50, 0.2)] = np.percentile(roll_std, 0.2)
        
    # change rate 
    dif = ad.diff().dropna().values
    
    change = (dif / ad.values[:-1])
    change = change[np.nonzero(change)[0]]
    change = change[~np.isnan(change)]
    change = change[change != -np.inf]
    change = change[change != np.inf]
    
    features[prefix + 'change_mean'] = change.mean()

    del change
    del dif
    
    train.append(features)
    
    del ad
    del chunk
    del real_ad
    del imag_ad
    del ft
    
X = pd.DataFrame(train)

In [ ]:
X = X[keep_cols]
X.head()

### Do thời gian chạy code chuyển đổi thông tin rất lâu, lưu lại dữ liệu đã xử lý

In [ ]:
X.to_csv("X_train.csv", index=False)
y.to_csv("y_train.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt 

plt.plot(range(len(y)), y)
plt.show()

Biểu đồ các giá trị của y. Ta thấy rằng do số mẫu rất nhỏ so với tổng số mẫu, nên tính liên tục của y không bị ảnh hưởng quá nhiều.

## Xây dựng dữ liệu từ bộ test

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv', index_col='seg_id')

test = []

for seg_id in tqdm(range(len(submission.index))):
    seg = pd.read_csv('../input/LANL-Earthquake-Prediction/test/' + submission.index[seg_id] + '.csv')

    ad = seg['acoustic_data']

    ft = np.fft.fft(ad)
    real_ad = pd.Series(np.real(ft))
    imag_ad = pd.Series(np.imag(ft))
    
    features = dict()
    
    # Tính các feature với ad
    #########################################################################################################
    quantiles = [0.5, 0.85]
    for quantile in quantiles:
        features['abs_quantile_' + str(quantile)] = np.quantile(np.abs(ad), quantile)
        
    div = 5
    for i in range(div):
        features['ave_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].mean()
        features['std_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].std()
        features['max_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].max()
        features['min_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].min()
        features['min_max_diff_div_' + str(i)] = features['max_div_' + str(i)] - features['min_div_' + str(i)]


    windows = [10, 50, 100, 200, 500, 1000, 5000, 10000]
    percentages = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99]
    for window in windows:
        roll_mean = ad.rolling(window).mean().dropna()
        roll_std = ad.rolling(window).std().dropna()

        features['rolling_min_std_' + str(window)] = roll_std.min()
        
        features['abs_rolling_max_mean_' + str(window)] = np.abs(roll_mean).max()
        
        for percentage in percentages:
            features['abs_rolling_mean_{}_percentage_{}'.format(window, percentage)] = np.percentile(roll_mean, percentage)
            features['abs_rolling_std_{}_percentage_{}'.format(window, percentage)] = np.percentile(roll_std, percentage)
            
    dif = ad.diff().dropna().values
    change = (dif / ad.values[:-1])
    change = change[np.nonzero(change)[0]]
    change = change[~np.isnan(change)]
    change = change[change != -np.inf]
    change = change[change != np.inf]
    
    features['change_mean'] = change.mean()
    
    features['hmean'] = stats.hmean(np.abs(ad[ad != 0]))
    features['gmean'] = stats.gmean(np.abs(ad[ad != 0]))
    
    features['iqr'] = np.subtract(*np.percentile(ad, [75, 25]))
    
    features['mad'] = np.mean(np.abs(ad - np.mean(ad)))
    
    del change
    del dif
    
    # Tính các feature với real_ad
    #########################################################################################################
    del ad
    ad = real_ad
    
    prefix = "real_"

    features[prefix + 'abs_min'] = np.abs(ad).min()
        
    features[prefix + 'abs_percentile_' + str(0.05)] = np.percentile(np.abs(ad), 0.05)
    features[prefix + 'abs_percentile_' + str(0.1)] = np.percentile(np.abs(ad), 0.1)
    features[prefix + 'abs_percentile_' + str(0.85)] = np.percentile(np.abs(ad), 0.85)
    
    # Tính các đặc tính theo tổng liên tiếp
    
    roll_mean = ad.rolling(10).mean().dropna()
    features[prefix + 'abs_rolling_max_mean_' + str(10)] = np.abs(roll_mean).max()
    roll_mean = ad.rolling(100).mean().dropna()
    features[prefix + 'abs_rolling_max_mean_' + str(100)] = np.abs(roll_mean).max()
    
    roll_mean = ad.rolling(10).mean().dropna()
    features[prefix + 'abs_rolling_mean_{}_percentage_{}'.format(10, 0.01)] = np.percentile(roll_mean, 0.01)
    roll_mean = ad.rolling(10000).mean().dropna()
    features[prefix + 'abs_rolling_mean_{}_percentage_{}'.format(10000, 0.15)] = np.percentile(roll_mean, 0.15)
    
    roll_std = ad.rolling(10).std().dropna()
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(10, 0.01)] = np.percentile(roll_std, 0.01)
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(10, 0.2)] = np.percentile(roll_std, 0.2)
    roll_std = ad.rolling(100).std().dropna()
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(100, 0.2)] = np.percentile(roll_std, 0.2)
    
    roll_mean = ad.rolling(10).mean().dropna()
    features[prefix + 'rolling_max_ave_' + str(10)] = roll_mean.max()
    roll_std = ad.rolling(5000).std().dropna()
    features[prefix + 'rolling_max_std_' + str(5000)] = roll_std.max()
            
    dif = ad.diff().dropna().values
    features[prefix + 'dif_mean'] = dif.mean()
    
    del dif
    
    # Tính các feature với imag_ad
    #########################################################################################################
    del ad
    ad = imag_ad
    
    prefix = "imag_"
    
    # find trend using linear regression.
    model = LinearRegression().fit(np.arange(len(ad)).reshape(-1, 1), ad)

    features[prefix + 'coef_'] = model.coef_[0]
    
    # divide into smaller segment.
    
    features[prefix + 'ave_div_' + str(2)] = ad[rows // div * 2: rows // div * (2 + 1)].mean()
    features[prefix + 'std_div_' + str(3)] = ad[rows // div * 3: rows // div * (3 + 1)].std()
    
    # some statistic distribution characteristic
    
    features[prefix + 'abs_percentile_' + str(0.1)] = np.percentile(np.abs(ad), 0.1)
    features[prefix + 'abs_percentile_' + str(0.15)] = np.percentile(np.abs(ad), 0.15)
        
    features[prefix + 'skew'] = stats.skew(ad)
    
    features[prefix + 'hann_window_' + str(5000)] = (convolve(ad, hann(5000), mode='same') / sum(hann(5000))).mean()
        
    roll_mean = ad.rolling(5000).mean().dropna()
    features[prefix + 'rolling_min_ave_' + str(5000)] = roll_mean.min()
    
    roll_mean = ad.rolling(200).mean().dropna()
    features[prefix + 'abs_rolling_min_mean_' + str(200)] = np.abs(roll_mean).min()
    
    roll_std = ad.rolling(10).std().dropna()
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(10, 0.01)] = np.percentile(roll_std, 0.01)
    
    roll_std = ad.rolling(50).std().dropna()
    features[prefix + 'abs_rolling_std_{}_percentage_{}'.format(50, 0.2)] = np.percentile(roll_std, 0.2)
        
    # change rate 
    dif = ad.diff().dropna().values
    
    change = (dif / ad.values[:-1])
    change = change[np.nonzero(change)[0]]
    change = change[~np.isnan(change)]
    change = change[change != -np.inf]
    change = change[change != np.inf]
    
    features[prefix + 'change_mean'] = change.mean()
    
    test.append(features)
    
X_test = pd.DataFrame(test)

In [ ]:
X_test = X_test[keep_cols]
X_test.head()

### Do thời gian chạy code chuyển đổi thông tin rất lâu, lưu lại dữ liệu đã xử lý (2)

In [ ]:
X_test.to_csv("X_test.csv", index=False)

In [ ]:
print(len(X_test))
print(len(X_test.columns))

# Kết quả

Dữ liệu đã xử lý được lưu trong [bộ dữ liệu](https://www.kaggle.com/ngodang/lanl-u-vo-chun-ha).